In [68]:
import os
import json
from typing import Dict, Any

In [69]:
def rearrange_based_on_list1(list1, list2):
    list1_keys = [extract_innermost(item) for item in list1]
    list2_keys = [extract_innermost(item) for item in list2]

    # Create a mapping from the innermost text of list2 to its items
    key_to_item = {key: item for item, key in zip(list2, list2_keys)}

    # Rearrange list2 based on the order of list1
    rearranged_list2 = []
    for key in list1_keys:
        if key in key_to_item:
            rearranged_list2.append(key_to_item[key])
            del key_to_item[key]  # Remove used key-item pair
        else:
            rearranged_list2.append("")  # Insert empty string if not present

    # Append remaining items from list2 to the end
    rearranged_list2.extend(key_to_item.values())

    return rearranged_list2


def extract_innermost(text):
    # Extract the innermost text within the parentheses
    while "(" in text:
        text = text.split("(", 1)[-1].rsplit(")", 1)[0]
    return text

In [70]:

def load_pddl_json(input_directory: str) -> Dict[str, Any]:
    with open(os.path.join(input_directory, "pddl-definitions.json")) as f:
        pddl_domain = json.load(f)
    pddl = {}
    for definition in pddl_domain["definitions"]:
        for verb in definition["actions"]:
            verb_pddl = {}
            verb_pddl["preconditions"] = [x for x in definition["preconditions"] if "?Y" not in x]
            verb_pddl["effects"] = [x for x in definition["effects"] if "?Y" not in x]
            verb_pddl["preconditions"] = sorted(verb_pddl["preconditions"], key=extract_innermost)
            verb_pddl["effects"] = rearrange_based_on_list1(verb_pddl["preconditions"], verb_pddl["effects"])
            pddl[verb] = verb_pddl
    return pddl


In [71]:
domain = load_pddl_json(input_directory="../pddl")
domain

{'put': {'preconditions': ['(in ?X hand)',
   '(is-holdable ?X)',
   '(not (onsurface ?X))',
   '(touching ?X hand)',
   '(within-reach ?X)'],
  'effects': ['(not (in ?X hand))',
   '(is-holdable ?X)',
   '(onsurface ?X)',
   '(not (touching ?X hand))',
   '(within-reach ?X)']},
 'drop': {'preconditions': ['(in ?X hand)',
   '(is-holdable ?X)',
   '(not (onsurface ?X))',
   '(touching ?X hand)',
   '(within-reach ?X)'],
  'effects': ['(not (in ?X hand))',
   '(is-holdable ?X)',
   '(onsurface ?X)',
   '(not (touching ?X hand))',
   '(within-reach ?X)']},
 'take': {'preconditions': ['(not (in ?X hand))',
   '(is-holdable ?X)',
   '(onsurface ?X)',
   '(not (touching ?X hand))',
   '(within-reach ?X)'],
  'effects': ['(in ?X hand)',
   '(is-holdable ?X)',
   '(not (onsurface ?X))',
   '(touching ?X hand)',
   '(within-reach ?X)']},
 'lift': {'preconditions': ['(not (in ?X hand))',
   '(is-holdable ?X)',
   '(onsurface ?X)',
   '(not (touching ?X hand))',
   '(within-reach ?X)'],
  'effec

In [72]:
def generate_pddl_from_dict(action_definitions):
    pddl_output = "; AUTO-GENERATED PDDL DOMAIN\n(define (domain egocentric-video)\n\t(:requirements :strips :typing :negative-preconditions)\n"

    # Adding actions
    for action, details in action_definitions.items():
        preconditions = " ".join(details["preconditions"])
        effects = " ".join(details["effects"])

        pddl_output += f"\t(:action {action}\n"
        pddl_output += "\t\t:parameters (?X)\n"  # Assuming a single parameter ?X for simplification
        pddl_output += f"\t\t:precondition (and {preconditions})\n"
        pddl_output += f"\t\t:effect (and {effects})\n"
        pddl_output += "\t)\n"

    pddl_output += ")"

    return pddl_output


with open("../pddl/filipo.pddl", "w") as f:
    f.write("\n".join(generate_pddl_from_dict(domain).split("\n")))

In [20]:
import sys

sys.path.append("../")

from src.pddl import Action, Predicate
from src.dataset import parse_pddl
import pandas as pd

domain, problem = parse_pddl(domain_path="../pddl/full_domain.pddl", problem_path="../pddl/problem.pddl")

verbs_path = "../data/epic-kitchens-100-annotations/EPIC_100_verb_classes.csv"
df = pd.read_csv(verbs_path, index_col=0)
all_verbs = sorted(df["key"].unique().tolist())

pddl_verbs = [action.name for action in domain]
pddl_verbs = sorted(pddl_verbs)
pddl_verbs

# print verbs in pddl_verbs but not in all_verbs
for verb in pddl_verbs:
    if verb not in all_verbs:
        print(verb)

# Print as a markdown list pddl_verbs
print("- " + "\n- ".join(pddl_verbs))

['add',
 'apply',
 'attach',
 'break',
 'close',
 'cut',
 'dry',
 'empty',
 'fill',
 'flip',
 'fold',
 'hold',
 'insert',
 'mix',
 'move',
 'open',
 'pat',
 'peel',
 'pour',
 'press',
 'put',
 'remove',
 'rip',
 'scoop',
 'scrape',
 'shake',
 'squeeze',
 'stretch',
 'take',
 'throw',
 'turn-off',
 'turn-on',
 'wash']